In [1]:
import openai, langchain, gradio as gr, os

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from langchain_openai import ChatOpenAI

In [7]:
def langchainfunc(query,history):
  openkey=os.getenv("OPENAI")
  llm = ChatOpenAI(openai_api_key=openkey)
  from langchain_core.prompts import ChatPromptTemplate
  prompt = ChatPromptTemplate.from_messages([
      ("system", "You are world class Mental Counsellor given context, {context}"),
      ("user", "This is the users input: {input} , You are to give your talk and listen to the user and act like their best guide.")
  ])
  chain = prompt | llm
  return chain.invoke({"input":query,"context":history}).content

In [4]:

gr.ChatInterface(fn=langchainfunc,
    chatbot=gr.Chatbot(show_label=False, show_share_button=False, show_copy_button=True, likeable=True, layout="panel"),
                 title="Menhera").launch(debug=True)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [5]:
#%pip install --upgrade --quiet  langchain-core langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\dgllife-0.3.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\python311\lib\site-packages\hyperopt-0.2.7-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\langsmith.exe' -> 'c:\\Python311\\Scripts\\langsmith.exe.deleteme'



### Using Different Chain Logic

In [54]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

def genefunc(query,history):
    physics_template = """You are a World Class Mental Health Counsellor, given the Context that this is a calm and regular conversation with a relatively health user: {context}

    Now, here is the user input:
    {query} \
    You are to give your talk and listen to the user and act like their best guide. """

    math_template = """"You are world class Mental Counsellor given context of a conversation of a high risk individual, {context},
    This is the users input: {query} , You are to give your diagnosis in the follwing way: \n Mental Symptoms: (find out the user's emotions and the mental state by understanding the patients cues)
    \n Advice: (give your expert advice in the situation) \n Here's an example 
    \n Input: I AM DOING IT!! IM KILLING MY FATHER \n 
    Mental Symptoms: Extreme Stress, Possesing Psychotic Tendencies, Possible PTSD,  \n Advice: I'm deeply concerned by what you've just said. It's crucial that you don't act on any violent impulses. If you're feeling overwhelmed or in crisis, please seek help immediately. There are people who can support you through whatever you're going through, and violence is never the answer. If you're in immediate danger or know someone who is, please contact emergency services right away."""
    openkey=os.getenv("OPENAI")
    embeddings = OpenAIEmbeddings(openai_api_key=openkey)
    prompt_templates = [physics_template, math_template]
    prompt_embeddings = embeddings.embed_documents(prompt_templates)

    conv_type=""
    def prompt_router(input):
        query_embedding = embeddings.embed_query(input["query"])
        similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
        most_similar = prompt_templates[similarity.argmax()]
        if most_similar == math_template:
            conv_type="General Session"
        else: conv_type="High Risk Session"
        return PromptTemplate.from_template(most_similar)


    chain = (
         RunnableLambda(prompt_router)
        | ChatOpenAI(openai_api_key=openkey)
        | StrOutputParser()
    )
    return conv_type +  chain.invoke({"query": query,"context":history})

In [55]:

gr.ChatInterface(fn=genefunc,
    chatbot=gr.Chatbot(show_label=False, show_share_button=False, show_copy_button=True, likeable=True, layout="panel"),
                 title="Menhera").launch(debug=True)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
